# Chapter 6 - Exercises

### 1. Set up a random experiment to test the difference between a sequential search and a binary search on a list of integers.

Let's first import the necesssary libraries and define both searching algorithms.

In [2]:
import time 
import random 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

def sequential_search(ls, e):
    for i, n in enumerate(ls):
        if e == n:
            return i 

    return -1

def binary_search(ls, e):
    low = 0
    high = len(ls) - 1

    while low <= high:
        mid = (low + high) // 2 
        if e < ls[mid]:
            high = mid - 1
        elif e > ls[mid]:
            low = mid + 1
        else:
            return mid

    return -1 
    

Let's now set up the experiment, by generating random ordered list (we need them for binary search, otherwise we would not get deterministic results). First let's start with values that really are inside the array.

In [10]:
n_simulations = 10
sizes = [10 ** i for i in range(1, 10)]

def get_stats_by_size(size, algorithm, times=1):
    access_times = []
    for _ in range(times):
        a = random.sample(range(size), size)
        a.sort()
        idx = random.randint(0, size - 1)
        e = a[idx]
        b = time.time()
        algorithm(a, e)
        e = time.time()
        access_times += [e - b]

    return access_times

def simulate(*algos, n_simulations=10):
    pass

[0.005635976791381836, 0.0007510185241699219, 0.0007317066192626953, 0.0006110668182373047, 0.0006308555603027344, 0.00017690658569335938, 0.0002491474151611328, 0.0004138946533203125, 0.0009541511535644531, 0.0003330707550048828, 0.0005199909210205078, 7.605552673339844e-05, 0.00040602684020996094, 9.202957153320312e-05, 8.821487426757812e-05, 0.0011272430419921875, 2.3126602172851562e-05, 0.0003790855407714844, 0.002306699752807617, 0.001238107681274414, 0.0008411407470703125, 1.9073486328125e-05, 0.0005908012390136719, 0.0006279945373535156, 0.0003399848937988281, 4.410743713378906e-05, 0.00043487548828125, 0.0003478527069091797, 0.0005640983581542969, 0.0005202293395996094, 0.00034999847412109375, 0.00021719932556152344, 0.00028228759765625, 8.893013000488281e-05, 0.00018095970153808594, 0.00010323524475097656, 5.3882598876953125e-05, 0.0008649826049804688, 0.0002989768981933594, 0.000640869140625, 0.00029015541076660156, 0.0005550384521484375, 0.0005891323089599609, 1.788139343261